In [1]:
import pickle
import os
%matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np


part_dict = ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear',\
             'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow',\
             'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee',\
             'right_knee', 'left_ankle', 'right_ankle']

COCO_dir = '/group/wanbo/HOI_plus/data/coco/images/train2014'


def showKps(kps):
    #kps: 51, show coco keypoints
    #kps = kps.reshape((17,3))
    v = kps[:,2]
    skeleton = [[16, 14], [14, 12], [17, 15], [15, 13], [12, 13],[6, 12], [7, 13], [6, 7], [6, 8],\
                [7, 9], [8, 10], [9, 11], [2, 3], [1, 2], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7]]

    for sk in skeleton:
        if v[sk[0]-1]>0 and v[sk[1]-1]>0:
            plt.plot([kps[sk[0]-1,0], kps[sk[1]-1,0]], [kps[sk[0]-1,1], kps[sk[1]-1,1]], 'r-')



In [1]:
import matplotlib.patches as patches

hoi_blob_path = '/root/backup/Research/HOI_plus/Outputs/fine_net3/debug/saved_hoi_blobs/'

hoi_blob_path = hoi_blob_path + 'saved_hoi_blobs3.pkl'
hoi_data= pickle.load(open(hoi_blob_path, 'rb'))
hoi_data = hoi_data[0]

im_id = hoi_data['id']
im_path = os.path.join(COCO_dir, 'COCO_train2014_'+str(im_id).zfill(12)+'.jpg')

print(os.path.isdir(COCO_dir))
#img = Image.open(im_path)
img = cv2.imread(im_path)
img = img[:,:,[2,1,0]]
plt.figure(figsize=(8,8))
if hoi_data['flipped']:
    img = img[:,::-1]
plt.imshow(img)

print (hoi_data.keys())
# for k, v in hoi_data.items():
#     if type(v)==np.ndarray:
#         print(k, v.shape)

#for i in range(union_boxes.shape[0]):
human_boxes = hoi_data['human_boxes'][hoi_data['interaction_human_inds']]
object_boxes = hoi_data['object_boxes'][hoi_data['interaction_object_inds']]

print('human_boxe.shape ', human_boxes.shape )
print('obj_boxes.shape ', object_boxes.shape)

union_boxes = hoi_data['union_boxes']
gt_union_heatmap = hoi_data['gt_union_heatmap']
union_mask = hoi_data['union_mask']

#print('union mask.shape:', union_mask.shape)
#print(human_boxes)
#print(object_boxes)

for i in range(union_boxes.shape[0]):
    plt.figure(figsize=(16,8))
    plt.subplot(241)
    plt.imshow(gt_union_heatmap[i,:17].sum(axis=0))
    plt.title('pose heatmap')
    
    pts = np.zeros((17,3))
    for j in range(17):
        y, x = np.unravel_index(np.argmax(gt_union_heatmap[i,j]), (64,64))
        pts[j,0] = x
        pts[j,1] = y
        pts[j,2] = np.max(gt_union_heatmap[i,j])
    
    plt.subplot(242)
    plt.imshow(gt_union_heatmap[i,17])
    plt.title('human proposal')
    
    plt.subplot(243)
    plt.imshow(gt_union_heatmap[i,18])
    plt.title('object proposal')
    
    x1,y1,x2,y2 = (union_boxes[i,1:]/hoi_data['image_info'][2]).astype(np.int32)
    
    hx1, hy1, hx2, hy2 = (human_boxes[i, 1:]/hoi_data['image_info'][2]).astype(np.int32)
    ox1, oy1, ox2, oy2 = (object_boxes[i, 1:]/hoi_data['image_info'][2]).astype(np.int32)
    
    scale_x = 64*1. / (x2-x1)
    scale_y = 64*1. / (y2-y1)
    #print (x1,y1,x2,y2)
    
    hx1 = ((hx1-x1)*scale_x).astype(np.int32)
    hx2 = ((hx2-x1)*scale_x).astype(np.int32)
    hy1 = ((hy1-y1)*scale_y).astype(np.int32)
    hy2 = ((hy2-y1)*scale_y).astype(np.int32)
    
    #print( hx1, hy1, hx2, hy2)
    ox1 = ((ox1-x1)*scale_x).astype(np.int32)
    ox2 = ((ox2-x1)*scale_x).astype(np.int32)
    oy1 = ((oy1-y1)*scale_y).astype(np.int32)
    oy2 = ((oy2-y1)*scale_y).astype(np.int32)
    #print(ox1, oy1, ox2, oy2)
    
    unimg = img[y1:y2, x1:x2, :]
    unimg = cv2.resize(unimg, (64,64))
    ax = plt.subplot(244)
    plt.imshow(unimg)
    showKps(pts)
    #print(pts[:,2])
    if np.max(pts[:,2])<1:
        plt.title('predicted pose')
    else:
        plt.title('gt pose')
    
    rect = patches.Rectangle((hx1, hy1), (hx2-hx1), (hy2-hy1),linewidth=3,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
    
    rect = patches.Rectangle((ox1, oy1), (ox2-ox1), (oy2-oy1),linewidth=3,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
    #plt.plot(pts[:,0], pts[:,1], 'ro')
    
    plt.subplot(245)
    plt.imshow(union_mask[i,0])
    plt.title('human proposal')
    
    plt.subplot(246)
    plt.imshow(union_mask[i,4])
    plt.title('union mask residual {}'.format(np.min(union_mask[i,4])))
    
    plt.subplot(247)
    plt.imshow(union_mask[i,2])
    plt.title('union mask ori')
    
    plt.subplot(248)
    plt.imshow(union_mask[i,3])
    plt.title('union mask noise ')
    
print(hoi_data['flipped'])


NameError: name 'pickle' is not defined

(21, 37)

In [14]:
print(img.size)
print(x1, y1, x2, y2)

(640, 427)
325 92 474 322


In [6]:
print(union_boxes.shape)

(16, 5)


In [26]:
hoi_data['image_info']

array([1069.       ,  800.       ,    1.6701461], dtype=float32)

In [32]:
print(hoi_data['union_boxes'].shape)
print(hoi_data['object_boxes'].shape)

(16, 5)
(15, 5)


In [28]:
1.67*640

1068.8

1.8735363